# 项目：欧洲国家足球分析

## 目录
<ul>
<li><a href="#intro">简介</a></li>
<li><a href="#wrangling">数据整理</a></li>
<li><a href="#eda">探索性数据分析</a></li>
<li><a href="#conclusions">结论</a></li>
</ul>

<a id='intro'></a>
## 简介

> 这个足球数据库来自 [Kaggle](https://www.kaggle.com/hugomathien/soccer)，它包含了从2008年到2016年间，几个欧洲国家的足球比赛（soccer matches）、球员（players）和球队（teams）的数据。该数据库存储在 SQLite 数据库中。

**该报告主要分析如下内容**
- 哪支球队在这段时间内的表现最好？ 
- 哪位球员受罚最多？

In [2]:
import numpy as np
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
% matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')

database = 'database.sqlite'

<a id='wrangling'></a>
## 数据整理


### 常规属性

In [3]:
#连接数据库
conn = sqlite3.connect(database)

In [4]:
#看一下都有哪些table
tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", conn)
tables

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,4,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,Player_Attributes,Player_Attributes,11,"CREATE TABLE ""Player_Attributes"" (\n\t`id`\tIN..."
2,table,Player,Player,14,CREATE TABLE `Player` (\n\t`id`\tINTEGER PRIMA...
3,table,Match,Match,18,CREATE TABLE `Match` (\n\t`id`\tINTEGER PRIMAR...
4,table,League,League,24,CREATE TABLE `League` (\n\t`id`\tINTEGER PRIMA...
5,table,Country,Country,26,CREATE TABLE `Country` (\n\t`id`\tINTEGER PRIM...
6,table,Team,Team,29,"CREATE TABLE ""Team"" (\n\t`id`\tINTEGER PRIMARY..."
7,table,Team_Attributes,Team_Attributes,2,CREATE TABLE `Team_Attributes` (\n\t`id`\tINTE...


In [5]:
# 得到DataFrame
countries = pd.read_sql("""SELECT *
                        FROM Country;""", conn)
leagues = pd.read_sql("""SELECT *
                        FROM League;""", conn)
matches = pd.read_sql("""SELECT *
                        FROM Match;""", conn)
players = pd.read_sql("""SELECT *
                        FROM Player;""", conn)
player_attributes = pd.read_sql("""SELECT *
                        FROM Player_Attributes;""", conn)
teams = pd.read_sql("""SELECT *
                        FROM Team;""", conn)
team_attributes = pd.read_sql("""SELECT *
                        FROM Team_Attributes;""", conn)

In [6]:
# 在Notebook中显示更多的行和列
pd.set_option('max_rows', 300)
pd.set_option('max_columns', 120)
pd.set_option('max_colwidth',1500)


### 根据Match、Country和League得到想要的DataFrame

In [13]:
detailed_matches = pd.read_sql("""SELECT Match.id, 
                                        Country.name AS country_name, 
                                        League.name AS league_name, 
                                        season, 
                                        stage, 
                                        date,
                                        HT.team_long_name AS  home_team,
                                        AT.team_long_name AS away_team,
                                        home_team_goal, 
                                        away_team_goal,
                                        CASE WHEN home_team_goal > away_team_goal THEN HT.team_long_name
                                             WHEN home_team_goal = away_team_goal THEN 'all_win'
                                             ELSE AT.team_long_name END AS winner
                                FROM Match
                                JOIN Country on Country.id = Match.country_id
                                JOIN League on League.id = Match.league_id
                                LEFT JOIN Team AS HT on HT.team_api_id = Match.home_team_api_id
                                LEFT JOIN Team AS AT on AT.team_api_id = Match.away_team_api_id
                                
                                ORDER by date
                                ;""", conn)
detailed_matches

,id,country_name,league_name,season,stage,date,home_team,away_team,home_team_goal,away_team_goal,winner
0,24559,Switzerland,Switzerland Super League,2008/2009,1,2008-07-18 00:00:00,BSC Young Boys,FC Basel,1,2,FC Basel
1,24560,Switzerland,Switzerland Super League,2008/2009,1,2008-07-19 00:00:00,FC Aarau,FC Sion,3,1,FC Aarau
2,24561,Switzerland,Switzerland Super League,2008/2009,1,2008-07-20 00:00:00,FC Luzern,FC Vaduz,1,2,FC Vaduz
3,24562,Switzerland,Switzerland Super League,2008/2009,1,2008-07-20 00:00:00,Neuchâtel Xamax,FC Zürich,1,2,FC Zürich
4,24613,Switzerland,Switzerland Super League,2008/2009,2,2008-07-23 00:00:00,FC Basel,Grasshopper Club Zürich,1,0,FC Basel
5,24614,Switzerland,Switzerland Super League,2008/2009,2,2008-07-23 00:00:00,AC Bellinzona,Neuchâtel Xamax,1,2,Neuchâtel Xamax
6,24615,Switzerland,Switzerland Super League,2008/2009,2,2008-07-23 00:00:00,FC Zürich,FC Luzern,1,0,FC Zürich
7,24616,Switzerland,Switzerland Super League,2008/2009,2,2008-07-24 00:00:00,FC Sion,BSC Young Boys,2,1,FC Sion
8,24617,Switzerland,Switzerland Super League,2008/2009,2,2008-07-24 00:00:00,FC Vaduz,FC Aarau,0,2,FC Aarau
9,24668,Switzerland,Switzerland Super League,2008/2009,3,2008-07-26 00:00:00,FC Basel,AC Bellinzona,2,0,FC Basel


<a id='eda'></a>
## 探索性数据分析


### 哪支球队在这段时间内的表现最好？

In [85]:
group_home = detailed_matches.groupby('home_team')
# group_home = detailed_matches.groupby(['home_team','league_name'])
all_goal_home1 = group_home.sum()['home_team_goal']
all_goal_home2 = group_home.sum()['away_team_goal']
all_goal_home = all_goal_home1 + all_goal_home2

group_away = detailed_matches.groupby('away_team')
# group_away = detailed_matches.groupby(['away_team', 'league_name'])
all_goal_away1 = group_away.sum()['home_team_goal']
all_goal_away2 = group_away.sum()['away_team_goal']
all_goal_away = all_goal_away1 + all_goal_away2

all_goal = all_goal_home + all_goal_away
all_goal.sort_values(ascending = False).head()
# all_goal.plot(kind='bar');
# type(all_goal)
# all_goal.index
# all_goal.plot(kind='bar',figsize=(150,40))

home_team
Real Madrid CF    1147
FC Barcelona      1081
PSV                946
FC Basel           930
SC Heerenveen      925
dtype: int64

> ###### _从上述程序可以看出，Real Madrid CF队总共进球1147个，表现最好_

***

### 哪位球员受罚最多？

In [79]:
player_attributes[['player_api_id','penalties']].sort_values(by='penalties',ascending=False)

,player_api_id,penalties
149594,39225,96.0
149593,39225,96.0
149592,39225,96.0
149591,39225,96.0
149595,39225,95.0
113528,108808,95.0
113521,108808,95.0
113522,108808,95.0
113523,108808,95.0
113524,108808,95.0


In [81]:
# 根据受罚最多的player_api_id得到球员姓名
players[players['player_api_id'] == 39225]

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
8967,8981,39225,Rickie Lambert,13879,1982-02-16 00:00:00,187.96,170


> ###### _根据penalties属性，球员Rickie Lambert受罚最多_

***

<a id='conclusions'></a>
## 结论

> 本分析是从进球数来判断一个球队是否表现好，可能还存在其他判断依据。Real Madrid CF是2008年到2016年间进球最多的一个球队。

> 在Player_Attributes这个表中，penalties列代表“处罚”之意，根据该列，Rickie Lambert这名球员所受到的处罚最多。